In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%cd /home/aditya/git/kaggle_pneumonia 
%env PROJECT_PATH = /home/aditya/git/kaggle_pneumonia 

/home/aditya/git/kaggle_pneumonia
env: PROJECT_PATH=/home/aditya/git/kaggle_pneumonia


In [3]:
%matplotlib inline
import seaborn as sns

In [4]:
import pandas as pd
import numpy as np
import pydicom
from PIL import Image
import multiprocessing as mp
from tqdm import tqdm_notebook as tqdm

In [5]:
from __future__ import print_function, division
import os
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import skimage
# Ignore warnings
import warnings
warnings.filterwarnings("ignore")

In [6]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import fastai
from fastai import *
from fastai.vision import *

In [7]:
from utils.envs import *
from utils.data_load import *
from utils.lr_finder import lr_plot, lr_find
from utils.common import get_batch_info
from utils.checkpoint import save_checkpoint, load_cp_model, load_cp_optim
from utils.logger import logger

In [8]:
from pathlib import Path
p = Path(data_source_path)

In [9]:
train_label_df = pd.read_csv(train_label_repo)
train_label_df['area'] = train_label_df.width * train_label_df.height
single_label_df = train_label_df.sort_values('area', ascending = False).drop_duplicates('patientId').sort_index().reset_index(drop = True).copy()
single_label_df.fillna(0, inplace = True)

In [10]:
device = torch.device('cuda:0')

In [ ]:
single_label_df.head(5)

In [ ]:
data = image_data_from_csv(p, 'train_images', csv_labels = 'label.csv', suffix='.png', valid_pct = 0.1, test='test_images')

In [ ]:
learn = ConvLearner(data, tvm.resnet18, metrics=accuracy)
learn.fit_one_cycle(1)

In [ ]:
x = learn.get_preds(is_test=True)

In [ ]:
log_preds = x[0].cpu().numpy()
preds = np.exp(log_preds)

In [ ]:
preds_prob = preds / np.sum(preds, axis = 1)[:, None]
submission = preds_prob[:, 1]

In [ ]:
test_path = data.test_dl.dataset.x


In [ ]:
##BBOX

In [11]:
md = image_data_from_csv(p, 'train_images', csv_labels = 'bb.csv', suffix='.png', valid_pct = 0.1, test='test_images', continuous = True)

In [12]:
bb_learn = ConvLearner(md, tvm.resnet34)

In [15]:
header = nn.Sequential(Flatten(), nn.Linear(25088,4)).to(device)
bb_learn = ConvLearner(md, tvm.resnet34, custom_head=header)
bb_learn.crit = nn.L1Loss().to(device)

In [ ]:
head_reg4 = nn.Sequential(Flatten(), nn.Linear(25088,4))
learn = ConvLearner.pretrained(f_model, md, custom_head=head_reg4)
learn.opt_fn = optim.Adam
learn.crit = nn.L1Loss()

In [16]:
bb_learn.fit_one_cycle(1)

RuntimeError: CUDA error: device-side assert triggered

In [17]:
bb_learn.get_preds(is_test=True)[0]

RuntimeError: CUDA error: device-side assert triggered